In [1]:
import time 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random
import datetime
import csv


In [9]:
from playwright.sync_api import sync_playwright



NameError: name 'page' is not defined

In [3]:
#new jupyter notebook to scrape the list of date and the corresponding website link
my_url_sample = 'http://www.nhc.gov.cn/xcs/yqjzqk/list_gzbd.shtml'


In [4]:
requests = requests.get(my_url_sample)
soup = BeautifulSoup(requests.text, 'html.parser')

[]